In [1]:
embed_model_name = "../models/bge-small-en-v1.5-strans"

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en-v1.5")

model.save(embed_model_name)

config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 380kB/s]
README.md: 100%|██████████| 94.8k/94.8k [00:00<00:00, 502kB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 144kB/s]
config.json: 100%|██████████| 743/743 [00:00<00:00, 8.05MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:12<00:00, 11.1MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 3.52MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 600kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.25MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.74MB/s]


## get documents

In [11]:
import requests

def get_wikipedia_page_links(title):
    S = requests.Session()

    URL = 'https://en.wikipedia.org/w/api.php'

    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'links',
        'titles': title,
        "pllimit": "max"
    }

    all_links = []

    while True:
        response = S.get(url=URL, params=params).json()
        pages = response.get('query', {}).get('pages', {})
        for page_id, page_content in pages.items():
            links = page_content.get('links', [])
            for link in links:
                all_links.append(link['title'])
        if 'continue' in response:
            params['plcontinue'] = response['continue']['plcontinue']
        else:
            break

    return all_links

In [14]:
links_lm = get_wikipedia_page_links("Large language model")
len(links_lm)

416

In [4]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("../dataset/llamaindex_data", glob="*", show_progress=True)
docs = loader.load()

 50%|█████     | 1/2 [00:07<00:07,  7.74s/it]This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to b

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500, 
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

documents = text_splitter.split_documents(docs)

In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 아무런 모델을 전달하지 않아도 임베딩 모델을 initialize할 수 있다.
# 다른 모델을 사용하고 싶으면 인자를 다르게 전달하여 불러올 수 있다.
embed_model = HuggingFaceEmbeddings()

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 886kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 47.0MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 379kB/s]
config.json: 100%|██████████| 571/571 [00:00<00:00, 5.88MB/s]
model.safetensors: 100%|██████████| 438M/438M [00:39<00:00, 11.1MB/s] 
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 538kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 598kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.18MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.44MB/s]


In [26]:
query = "what is deep learning?"
embed_model.embed_query(query)[:10]

[-0.023704713210463524,
 -0.002275315811857581,
 -0.044309794902801514,
 -0.005287010222673416,
 -0.011042586527764797,
 0.032786980271339417,
 0.006997972726821899,
 0.0012319738743826747,
 0.002887121867388487,
 0.004598310217261314]

In [31]:
# 허깅페이스에서 특정 임베딩 모델을 로드
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [32]:
query = "what is deep learning?"
embed_model.embed_query(query)[:10]

[-0.0077447895891964436,
 -0.02521646022796631,
 -0.024298133328557014,
 -0.005299137905240059,
 0.05355464294552803,
 0.0639205276966095,
 0.041832778602838516,
 -0.0223032608628273,
 0.0807005986571312,
 0.0017157538095489144]

In [33]:
# 로컬에서 임베딩 모델을 불러오는 경우에는 모델의 경로를 인자로 전달하면 된다.
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [34]:
query = "what is deep learning?"
embed_model.embed_query(query)[:10]

[-0.0077447895891964436,
 -0.02521646022796631,
 -0.024298133328557014,
 -0.005299137905240059,
 0.05355464294552803,
 0.0639205276966095,
 0.041832778602838516,
 -0.0223032608628273,
 0.0807005986571312,
 0.0017157538095489144]

In [36]:
# bge model을 사용할 때는 HuggingFaceBgeEmbeddings를 사용할 수 있다.
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

embed_model = HuggingFaceBgeEmbeddings(
    model_name=embed_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [37]:
query = "what is deep learning?"
embed_model.embed_query(query)[:10]

[-0.00322693376801908,
 -0.01634078659117222,
 -0.03721240535378456,
 0.012990396469831467,
 0.05193884298205376,
 0.058871809393167496,
 0.03991677984595299,
 -0.021628687158226967,
 0.07608047872781754,
 -0.01793438196182251]

#### HuggingFaceEmbedding과 결과가 다른 이유?

* `HuggingFaceEmbedding`
```python
def embed_documents(self, texts: List[str]) -> List[List[float]]:
    import sentence_transformers

    texts = list(map(lambda x: x.replace("\n", " "), texts))
    if self.multi_process:
        pool = self.client.start_multi_process_pool()
        embeddings = self.client.encode_multi_process(texts, pool)
        sentence_transformers.SentenceTransformer.stop_multi_process_pool(pool)
    else:
        embeddings = self.client.encode(
            texts, show_progress_bar=self.show_progress, **self.encode_kwargs
        )

    return embeddings.tolist()

def embed_query(self, text: str) -> List[float]:

    return self.embed_documents([text])[0]
```

* `HuggingFaceBgeEmbeddings`
```python
def embed_query(self, text: str) -> List[float]:
    text = text.replace("\n", " ")
    embedding = self.client.encode(
        self.query_instruction + text, **self.encode_kwargs
    )
    return embedding.tolist()

```

- `HuggingFaceBgeEmbeddings`의 경우에는 임베딩을 진행할 때 query_intruction과 query를 결합하는 과정을 거친다.
- `HuggingFaceEmbeddings`는 이러한 query_instruction을 결합하는 과정이 들어있지 않기 때문에 두 임베딩 결과의 차이가 존재하는 것이다.
- query_intruction은 "Represent this quetion for searching relevant passages:"이며 이를 query 앞에 붙인다면 `HuggingFaceEmbeddings 또한 동일한 임베딩 결과가 나올 것이다.

In [45]:
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [46]:
query = "Represent this question for searching relevant passages: What is deep learning?"
embed_model.embed_query(query)[:10]

[-0.00322693376801908,
 -0.01634078659117222,
 -0.03721240535378456,
 0.012990396469831467,
 0.05193884298205376,
 0.058871809393167496,
 0.03991677984595299,
 -0.021628687158226967,
 0.07608047872781754,
 -0.01793438196182251]

In [47]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

embed_model = HuggingFaceBgeEmbeddings(
    model_name=embed_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [48]:
query = "What is deep learning?"
embed_model.embed_query(query)[:10]

[-0.00322693376801908,
 -0.01634078659117222,
 -0.03721240535378456,
 0.012990396469831467,
 0.05193884298205376,
 0.058871809393167496,
 0.03991677984595299,
 -0.021628687158226967,
 0.07608047872781754,
 -0.01793438196182251]

# Retrieval

In [49]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embed_model)

In [50]:
retriever = vector_index.as_retriever(serach_type="mmr")

In [51]:
retrieved = retriever.get_relevant_documents("Waht country is Mistral AI based in?")

In [54]:
retrieved[0].page_content

'24. 4. 10. 오후 3:22\n\nMistral AI - Wikipedia\n\nWIKIPEDIA\n\nThe Free Encyclopedia\n\nMistral AI\n\nMistral AI is a French company selling artificial intelligence (AI) products. It was founded in April 2023 by previous employees of Meta Platforms and Google DeepMind.[1] The company raised €385 million in October 2023[2] and in December 2023 it was valued at more than $2 billion.[3][4][5]\n\nMistral AI'

# Local LLM

In [61]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("Writer/camel-5b-hf")
model = AutoModel.from_pretrained("Writer/camel-5b-hf")

tokenizer.save_pretrained("../models/camel-5b-hf")
model.save_pretrained("../models/camel-5b-hf")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/Users/choi/Desktop/rag-with-llamaindex/venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [01:03<00:00, 21.14s/it]


In [62]:
model_name = "../models/camel-5b-hf"

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("../models/camel-5b-hf")
model = AutoModelForCausalLM.from_pretrained("../models/camel-5b-hf")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

/Users/choi/Desktop/rag-with-llamaindex/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:57<00:00, 11.41s/it]


In [ ]:
pipe("Waht is OpenAI Sora")

In [ ]:
# 허깅페이스의 pipeline을 그대로 chain에 사용하면 랭체인의 문법과 맞지 않아 오류가 발생한다.
from langchain.prompts import PromptTemplate

template = """Quation: {quetion}
Answer: """

prompt = PromptTemplate(template)

chain = prompt | pipe

In [ ]:
chain.invoke({"quetion" : "What is OpenAI Sora?"})

In [ ]:
# 랭체인에서 제공하는 파이프라인으로 한번 감싸줘야 오류가 발생하지 않는다.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline(pipeline=pipe)

chain = prompt | hf

In [ ]:
chain.invoke({"quetion" : "What is OpenAI Sora?"})

In [ ]:
# 파이프라인을 먼저 만들지 않고 바로 HuggingFacePipeline을 통해서 파이프라인을 불러올 수 있다.
hf = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task='text-generation',
    pipleline_kwargs={'max_new_tokens': 512}
)

In [ ]:
chain = prompt | hf

In [ ]:
chain.invoke({"quetion" : "What is OpenAI Sora?"})

# Putting it all together

In [1]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
def merge_docs(retrieved_docs):
    return "\n\n".join([d.page_content for d in retrieved_docs])

loader = DirectoryLoader("../dataset/llamaindex_data", glob="*", show_progress=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

documents = text_splitter.split_documents(docs)

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:02<00:02,  2.42s/it]This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to b

In [3]:
embed_model_name = "../models/bge-small-en-v1.5-strans"
model_name = "../models/camel-5b-hf"

In [4]:
embed_model = HuggingFaceBgeEmbeddings(
    model_name=embed_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

In [5]:
vector_index = FAISS.from_documents(documents, embed_model)
retriever = vector_index.as_retriever(search_type="mmr")

In [6]:
hf = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 512}
)

Loading checkpoint shards: 100%|██████████| 5/5 [00:49<00:00,  9.92s/it]


In [7]:
template = """
Utilizing the context given below, answer the question
[context]
{context}

question: {query}
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
chain = RunnableParallel({"context": retriever|merge_docs, "query": RunnablePassthrough()}) \
        | prompt \
        | hf \
        | StrOutputParser()

In [9]:
chain.invoke("What is OpenAI sora")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

# vector index save & load

In [ ]:
vector_index.save_local("../models/faiss_local.json")

In [ ]:
vector_index = FAISS.load_local("../models/faiss_local.json", embeddings=embed_model, allow_dangerous_deserialization=True)

In [ ]:
retriever = vector_index.as_retriver(search_type="mmr")
retreived = retriever.get_relevant_documents("What county is Mistral AI based in?")
retrieved[0].page_content